# Install librarys
Install packages need to train mario agent.

In [ ]:
# !pip install torch
# !pip install numpy
# !pip install matplotlib
!pip install gymnasium==0.29.1
!pip install gym-super-mario-bros==7.4.0
!pip install gym==0.25.2
!pip install imageio-ffmpeg
!pip install imageio
# !pip install torchvision
!pip install opencv-python-headless

# import packages

In [ ]:
from PIL import Image
from collections import deque
from datetime import datetime
from pathlib import Path
import copy
import cv2
import imageio
import numpy as np
import random, os
import torch
from torch import nn
import torch.nn.functional as F
import torch.multiprocessing as mp
#import multiprocessing as mp
from torchvision import transforms as T

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY

# Create hyperparammeters
Config hyperparammeters, just change it

In [ ]:
#class DictWrapper create by Chatgpt
class DictWrapper:
    def __init__(self, dictionary):
        self._dict = dictionary

    def __getattr__(self, item):
        if item in self._dict:
            return self._dict[item]
        else:
            raise AttributeError(f"'DictWrapper' object has no attribute '{item}'")

config = {
    'num_envs': 16,
    'save_model_step': int(1e5),
    'save_figure_step': 512,
    'learn_step': 512,
    'total_step_or_episode': 'step',
    'total_step': int(5e6),
    'total_episode': None,
    'batch_size': 64,
    'save_dir': "",
    'gamma': 0.9,
    'learning_rate': 7e-5,
    'state_dim': (4, 84, 84),
    'action_dim': 12,#12 for complex, 7 for simple
    'entropy_coef': 0.01,
    'V_coef': 0.5,
    'max_grad_norm': 0.5,
    'clip_param': 0.2,
    'num_epoch': 2,
    'world': 1,
    'stage': 1,
    'action_type': 'complex',
    'is_normalize_advantage': False,
    'V_loss_type': "huber", #"mse"
    'target_kl': 0.05,
    'gae_lambda': 0.95,
    'per_buffer_size': 256,
    'per_eps': 1e-2,
    'per_alpha': 0.7,
    'per_beta': 0.4,
    'eps_marg': 0.2,
    'off_epoch': 8,
}

config = DictWrapper(config)

In [ ]:
gym.__version__

# Define environment
## Create a custom environment, We need:
- SkipFrame: because the episode is very long then in this environment, we only need to repeat action sometimes. Then we repeat each action 4 times and skip 3 first frames (return 4th frame). We need to sum 4 rewards from 4 frames.
- GrayScaleResizeObservation: grayscale state (from RGB to gray image) and resize to 84x84.
- NoopResetEnv: when resetting the environment, I do random actions sometimes before using the evironment. This same as atari strategy. When reset, we choice num_noops random actions by random from 0 to noop_max and random do num_noops actions. If random actions yield to a terminal state, then reset and continue do random action. I set noop_max as 30 same as atari.
- CustomRewardAndDoneEnv
    - I see that a lot of people train Mario to use this custom reward, and then I copy it. Just add 50 rewards if the agent solves the stage and add -50 rewards if the agent is dead. And the reward is divided into 10. I set done to True if Mario dies instead of when Mario lost all lives as default.
    - stage 4-2: give a -50 reward when Mario moves on top of the map (y_pos >= 255).
    - stages 4-4 and 7-4: set done = True when Mario goes to wrong way and gives a -50 penalty reward. If Mario goes to the true way but the map still loops (this is a bug), I set done = True but don't give a penalty reward.
    - stage 8-4: set done = True when Mario goes to wrong way and gives a -50 penalty reward (like 4-4 and 7-4), but this map have 1 hardest part that Mario need to find hidden brick. I set penalty reward as -100 for this part (info["x_pos"] > 2440 and info["x_pos"] <= 2500).
    - stages 4-4 and 8-4: give a -0.1 reward every step.


## About reward system:
- set done to True when Mario dies: This is the most important thing because, in the default reward system, Mario still yields a reward by just moving right, if Mario dies, the agent can't lose total rewards and the agent still go right (in new life) and get more rewards. This is the easiest way to get rewards and the agent also learns this trick.
- penalty -50 reward when Mario dies: this is necessary to make Mario train faster. If we don't add a penalty, the agent can't complete hard stages.
reward 50 when reaching the flag: this makes Mario train faster and surpass stuck points in hard stages.
- change penalty and reaching flag reward of more or less than 50 don't make more difference then I don't change it.
- Divide rewards by 10: I think it makes total rewards smaller and the agent can learn a better way, but I am not sure how it is necessary. I just copy it.
- With Stage 4-2: I see that the agent can reach more rewards when Mario goes to the wrap zone, but Mario can't win this stage with the wrap zone way because the reward system gives negative rewards when Mario goes left. Then I just give a penalty reward when Mario moves to the top of the map.
- Use FrameStack to stack lastest 4 frames as observation
- with stage 4-4 and 7-4:
    - because this map has the wrong way, Mario will go to the loop and the reward still increase to infinity, then I need to set done = True and give a negative penalty reward
    - negative reward to prevent Mario go to the wrong way.
    - Another strategy is to give a negative reward but don't set done = True (like 4-2). But this map has a bug and this strangtegy not working.
    - even though Mario goes in the correct way, sometimes Mario still goes in the loop. Then I set done = True every time Mario goes on the loop (check by x_pos and max_x_pos).
- stage 4-4: give a -0.1 reward every step: prevent Mario stuck. This map have 1 part that Mario need to move left, but move left give negative reward in default system. Go right make Mario go to wrong way, than the episode ended and Mario get negative reward. Than Mario often stop moving to maintain rewards. To make Mario move, I need add negative reward every step.
- stage 8-4:
    - give a -0.1 reward every step: prevent Mario from getting stuck. This makes Mario move when they are stuck at part (info["x_pos"] > 2440 and info["x_pos"] <= 2500). Because if Mario can't move right (go the wrong way and get negative reward), Agent often learn that they can't do anything to get more reward. Then agent often stop moving. This penalty reward makes agent still explore to find hidden brick.
    - same as 4-4 and 7-4, I set done = True and give -50 penalty reward when Agent move in wrong way. But at hardest part (info["x_pos"] > 2440 and info["x_pos"] <= 2500), I set penalty reward as -100.
    - Another strategy is giving +50 reward when agent find hidden brick or agent go in true way (don't go the wrong way). I tried and all 3 ways work.
    - Because the map has many locations with overlapping x_pos (especially the underwater part where x_pos is marked again from 1). You must be careful if changing the custom reward system.

In [ ]:
# Initialize Super Mario environment (in v0.26 change render mode to 'human' to see results on the screen)
if gym.__version__ < '0.26':
    env = gym_super_mario_bros.make(f"SuperMarioBros-{config.world}-{config.stage}-v0", new_step_api=True)
else:
    env = gym_super_mario_bros.make(f"SuperMarioBros-{config.world}-{config.stage}-v0", render_mode='rgb', apply_api_compatibility=True)

env = JoypadSpace(env, COMPLEX_MOVEMENT)
print(env.action_space)

env.reset()
next_state, reward, done, trunc, info = env.step(action=0)
print(f"{next_state.shape},\n {reward},\n {done},\n {info}")

class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, trunk, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, trunk, info

class GrayScaleResizeObservation(gym.ObservationWrapper):
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int):
            self.shape = (shape, shape)
        else:
            self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        self.current_state = observation
        observation = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
        observation = cv2.resize(observation, self.shape, interpolation=cv2.INTER_AREA)
        observation = observation.astype(np.uint8)#.reshape(-1, observation.shape[0], observation.shape[1])
        return observation

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        super(NoopResetEnv, self).__init__(env)
        self.noop_max = noop_max

    def reset(self, **kwargs):
        """Do no-op action for a number of steps in [1, noop_max]."""
        obs = self.env.reset(**kwargs)
        noops = np.random.randint(0, self.noop_max, (1, ))[0]
        for _ in range(noops):
            action = self.env.action_space.sample()
            obs, _, done, _, _ = self.env.step(action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        obs, reward, done, trunk, info = self.env.step(ac)
        return obs, reward, done, trunk, info

class CustomRewardAndDoneEnv(gym.Wrapper):
    def __init__(self, env=None, world=1, stage=1):
        super(CustomRewardAndDoneEnv, self).__init__(env)
        self.current_score = 0
        self.current_x = 0
        self.current_x_count = 0
        self.max_x = 0
        self.world = world
        self.stage = stage
        if self.world == 8 and self.stage == 4:
            self.sea_map = False

    def reset(self, **kwargs):
        self.current_score = 0
        self.current_x = 0
        self.current_x_count = 0
        self.max_x = 0
        if self.world == 8 and self.stage == 4:
            self.sea_map = False
        return self.env.reset(**kwargs)

    def step(self, action):
        state, reward, done, trunc, info = self.env.step(action)

        if (info['x_pos'] - self.current_x) == 0:
            self.current_x_count += 1
        else:
            self.current_x_count = 0
        if info["flag_get"]:
            reward += 50
            done = True
        if done and info["flag_get"] == False and info["time"] != 0:
            reward -= 50
            done = True
        self.current_x = info["x_pos"]

        if self.world == 7 and self.stage == 4:
            if (506 <= info["x_pos"] <= 832 and info["y_pos"] > 127) or (
                    832 < info["x_pos"] <= 1064 and info["y_pos"] < 80) or (
                    1113 < info["x_pos"] <= 1464 and info["y_pos"] < 191) or (
                    1579 < info["x_pos"] <= 1943 and info["y_pos"] < 191) or (
                    1946 < info["x_pos"] <= 1964 and info["y_pos"] >= 191) or (
                    1984 < info["x_pos"] <= 2060 and (info["y_pos"] >= 191 or info["y_pos"] < 127)) or (
                    2114 < info["x_pos"] < 2440 and info["y_pos"] < 191):
                reward -= 50
                done = True
            if done == False and info["x_pos"] < self.max_x - 100:
                done = True
        if self.world == 4 and self.stage == 4:
            if (info["x_pos"] <= 1500 and info["y_pos"] < 127) or (
                    1588 <= info["x_pos"] < 2380 and info["y_pos"] >= 127):
                reward = -50
                done = True
            if done == False and info["x_pos"] < self.max_x - 100:
                done = True
            if done == False:
                reward -= 0.1
        if self.world == 4 and self.stage == 2 and done == False and info['y_pos'] >= 255:
            reward -= 50
        if self.world == 8 and self.stage == 4:
            if info["x_pos"] > 2440 and info["x_pos"] <= 2500:
                done = True
                reward -= 100
            if info["x_pos"] >= 3675 and info["x_pos"] <= 3700:
                done = True
                reward -= 50

            if info["x_pos"] < self.max_x - 200:
                if self.max_x >= 1250 and self.max_x <= 1310: #solved bug because x_pos duplicated
                    if info["x_pos"] >= 320:
                        done = True
                        reward = -50
                elif info["x_pos"] == 312:
                    done = True
                    reward = -50
                elif info["x_pos"] == 56 and self.max_x > 3650 and self.sea_map == False:
                    reward += 50
                    self.sea_map = True
            if info["x_pos"] > self.max_x + 100:
                reward += 50
            if done == False:
                reward -= 0.1
        self.max_x = max(self.max_x, self.current_x)
        self.current_score = info["score"]

        return state, reward / 10., done, trunc, info

# Create MultipleEnvironments
MultipleEnvironments use multi-processing to parallel running.

Because in the training process, we need to reset the environment when the agent reaches the terminal state. But if we will do it in parallel, then I don't want to check each environment and reset (by loop) or create a new function that parallels check and reset all environments. Then I reset the environment if done = True in step function and set next_state = env.reset(). Then in training, we just set state = next_state (next_state is reset state if done = True)

In [ ]:
#modify from https://github.com/uvipen/Super-mario-bros-PPO-pytorch/blob/master/src/env.py
def create_env(world, stage, action_type, test=False):
    if gym.__version__ < '0.26':
        env = gym_super_mario_bros.make(f"SuperMarioBros-{world}-{stage}-v0", new_step_api=True)
    else:
        env = gym_super_mario_bros.make(f"SuperMarioBros-{world}-{stage}-v0", render_mode='rgb', apply_api_compatibility=True)

    if action_type == "right":
        action_type = RIGHT_ONLY
    elif action_type == "simple":
        action_type = SIMPLE_MOVEMENT
    else:
        action_type = COMPLEX_MOVEMENT

    env = JoypadSpace(env, action_type)

    if test == False:
        env = NoopResetEnv(env)
    env = SkipFrame(env, skip=4)
    env = CustomRewardAndDoneEnv(env, world, stage)
    env = GrayScaleResizeObservation(env, shape=84)
    if gym.__version__ < '0.26':
        env = FrameStack(env, num_stack=4, new_step_api=True)
    else:
        env = FrameStack(env, num_stack=4)
    return env

class MultipleEnvironments:
    def __init__(self, world, stage, action_type, num_envs):
        self.agent_conns, self.env_conns = zip(*[mp.Pipe(duplex=True) for _ in range(num_envs)])
        self.envs = [create_env(world, stage, action_type) for _ in range(num_envs)]

        for index in range(num_envs):
            process = mp.Process(target=self.run, args=(index,))
            process.start()
            self.env_conns[index].close()

    def run(self, index):
        self.agent_conns[index].close()
        while True:
            request, action = self.env_conns[index].recv()
            if request == "step":
                next_state, reward, done, trunc, info = self.envs[index].step(action)
                if done:
                    next_state = self.envs[index].reset()
                self.env_conns[index].send((next_state, reward, done, trunc, info))
            elif request == "reset":
                self.env_conns[index].send(self.envs[index].reset())
            else:
                raise NotImplementedError

    def step(self, actions):
        [agent_conn.send(("step", act)) for agent_conn, act in zip(self.agent_conns, actions)]
        next_states, rewards, dones, truncs, infos = zip(*[agent_conn.recv() for agent_conn in self.agent_conns])
        return next_states, rewards, dones, truncs, infos

    def reset(self):
        [agent_conn.send(("reset", None)) for agent_conn in self.agent_conns]
        states = [agent_conn.recv() for agent_conn in self.agent_conns]
        return states

In [ ]:
mp.cpu_count()

# Create memory
Memory just save all info we need to train and return all stored info

In [ ]:
class Memory():
    def __init__(self, num_envs):
        self.num_envs = num_envs

        self.states = []
        self.actions = []
        self.next_states = []
        self.rewards = []
        self.dones = []
        self.logits = []
        self.values = []

    def save(self, state, action, reward, next_state, done, logit, V):
        self.states.append(state)
        self.actions.append(action)
        self.next_states.append(next_state)
        self.rewards.append(reward)
        self.dones.append(done)
        self.logits.append(logit)
        self.values.append(V)

    def reset(self):
        self.states = []
        self.actions = []
        self.next_states = []
        self.rewards = []
        self.dones = []
        self.logits = []
        self.values = []

    def get_data(self):
        return self.states, self.actions, self.next_states, self.rewards, self.dones, self.logits, self.values

## SumTree
I copy from [Howuhh prioritized_experience_replay](https://github.com/Howuhh/prioritized_experience_replay/blob/main/memory/buffer.py)

In [ ]:
# The ‘sum-tree’ data structure used here is very similar in spirit to the array representation
# of a binary heap. However, instead of the usual heap property, the value of a parent node is
# the sum of its children. Leaf nodes store the transition priorities and the internal nodes are
# intermediate sums, with the parent node containing the sum over all priorities, p_total. This
# provides a efficient way of calculating the cumulative sum of priorities, allowing O(log N) updates
# and sampling. (Appendix B.2.1, Proportional prioritization)

# Additional useful links
# Good tutorial about SumTree data structure:  https://adventuresinmachinelearning.com/sumtree-introduction-python/
# How to represent full binary tree as array: https://stackoverflow.com/questions/8256222/binary-tree-represented-using-array
class SumTree:
    def __init__(self, size):
        self.nodes = [0] * (2 * size - 1)
        self.data = [None] * size

        self.size = size
        self.count = 0
        self.real_size = 0

    @property
    def total(self):
        return self.nodes[0]

    def update(self, data_idx, value):
        idx = data_idx + self.size - 1  # child index in tree array
        change = value - self.nodes[idx]

        self.nodes[idx] = value

        parent = (idx - 1) // 2
        while parent >= 0:
            self.nodes[parent] += change
            parent = (parent - 1) // 2

    def add(self, value, data):
        self.data[self.count] = data
        self.update(self.count, value)

        self.count = (self.count + 1) % self.size
        self.real_size = min(self.size, self.real_size + 1)

    def get(self, cumsum):
        assert cumsum <= self.total

        idx = 0
        while 2 * idx + 1 < len(self.nodes):
            left, right = 2*idx + 1, 2*idx + 2

            if cumsum <= self.nodes[left]:
                idx = left
            else:
                idx = right
                cumsum = cumsum - self.nodes[left]

        data_idx = idx - self.size + 1

        return data_idx, self.nodes[idx], self.data[data_idx]

    def __repr__(self):
        return f"SumTree(nodes={self.nodes.__repr__()}, data={self.data.__repr__()})"

## PrioritizedReplayBuffer

I edit from [Howuhh prioritized_experience_replay](https://github.com/Howuhh/prioritized_experience_replay/blob/main/memory/buffer.py)

In [ ]:
class PrioritizedReplayBuffer:
    def __init__(self, learn_step, state_dim, num_action, buffer_size, eps=1e-2, alpha=0.1, beta=0.1):
        self.tree = SumTree(size=buffer_size)

        # PER params
        self.eps = eps  # minimal priority, prevents zero probabilities
        self.alpha = alpha  # determines how much prioritization is used, α = 0 corresponding to the uniform case
        self.beta = beta  # determines the amount of importance-sampling correction, b = 1 fully compensate for the non-uniform probabilities
        self.max_priority = eps  # priority for new samples, init as eps

        # transition: state, action, reward, next_state, done
        self.states = np.zeros((buffer_size, learn_step, *state_dim), dtype = np.uint8)
        self.actions = np.zeros((buffer_size, learn_step))
        self.next_states = np.zeros((buffer_size, learn_step, 1, state_dim[1], state_dim[2]), dtype = np.uint8)
        self.rewards = np.zeros((buffer_size, learn_step), dtype = np.float32)
        self.dones = np.zeros((buffer_size, learn_step))
        self.b_logits = np.zeros((buffer_size, learn_step, num_action), dtype = np.float32)

        self.count = 0
        self.real_size = 0
        self.size = buffer_size

    def add(self, transition, priority):
        state, action, reward, next_state, done, b_logit = transition

        # store transition index with maximum priority in sum tree

        self.tree.add(self.max_priority, self.count)
        self.update_priorities([self.count], [priority])

        # store transition in the buffer
        self.states[self.count] = state
        self.actions[self.count] = action
        self.rewards[self.count] = reward
        next_state = next_state[:, 3]
        next_state = next_state.reshape(next_state.shape[0], 1, next_state.shape[1], next_state.shape[2])
        self.next_states[self.count] = next_state
        self.dones[self.count] = done
        self.b_logits[self.count] = b_logit

        # update counters
        self.count = (self.count + 1) % self.size
        self.real_size = min(self.size, self.real_size + 1)

    def sample(self, batch_size):
        assert self.real_size >= batch_size, "buffer contains less samples than batch size"

        sample_idxs, tree_idxs = [], []
        priorities = np.zeros((batch_size,))

        # To sample a minibatch of size k, the range [0, p_total] is divided equally into k ranges.
        # Next, a value is uniformly sampled from each range. Finally the transitions that correspond
        # to each of these sampled values are retrieved from the tree. (Appendix B.2.1, Proportional prioritization)
        segment = self.tree.total / batch_size
        for i in range(batch_size):
            a, b = segment * i, segment * (i + 1)

            cumsum = random.uniform(a, b)
            # sample_idx is a sample index in buffer, needed further to sample actual transitions
            # tree_idx is a index of a sample in the tree, needed further to update priorities
            tree_idx, priority, sample_idx = self.tree.get(cumsum)

            priorities[i] = priority
            tree_idxs.append(tree_idx)
            sample_idxs.append(sample_idx)

        # Concretely, we define the probability of sampling transition i as P(i) = p_i^α / \sum_{k} p_k^α
        # where p_i > 0 is the priority of transition i. (Section 3.3)
        probs = priorities / self.tree.total

        # The estimation of the expected value with stochastic updates relies on those updates corresponding
        # to the same distribution as its expectation. Prioritized replay introduces bias because it changes this
        # distribution in an uncontrolled fashion, and therefore changes the solution that the estimates will
        # converge to (even if the policy and state distribution are fixed). We can correct this bias by using
        # importance-sampling (IS) weights w_i = (1/N * 1/P(i))^β that fully compensates for the non-uniform
        # probabilities P(i) if β = 1. These weights can be folded into the Q-learning update by using w_i * δ_i
        # instead of δ_i (this is thus weighted IS, not ordinary IS, see e.g. Mahmood et al., 2014).
        # For stability reasons, we always normalize weights by 1/maxi wi so that they only scale the
        # update downwards (Section 3.4, first paragraph)
        weights = (self.real_size * probs) ** -self.beta

        # As mentioned in Section 3.4, whenever importance sampling is used, all weights w_i were scaled
        # so that max_i w_i = 1. We found that this worked better in practice as it kept all weights
        # within a reasonable range, avoiding the possibility of extremely large updates. (Appendix B.2.1, Proportional prioritization)
        weights = weights / weights.max()

        states = self.states[sample_idxs]
        next_states = self.next_states[sample_idxs]
        next_states = np.concatenate((states[:, :, :3, :, :], next_states), axis = 2)

        batch = (
            states.transpose(1, 0, 2, 3, 4),
            self.actions[sample_idxs].transpose(1, 0),
            next_states.transpose(1, 0, 2, 3, 4),
            self.rewards[sample_idxs].transpose(1, 0),
            self.dones[sample_idxs].transpose(1, 0),
            self.b_logits[sample_idxs].transpose(1, 0, 2)
        )
        return batch, weights, tree_idxs

    def update_priorities(self, data_idxs, priorities):
        if isinstance(priorities, torch.Tensor):
            priorities = priorities.detach().cpu().numpy()

        for data_idx, priority in zip(data_idxs, priorities):
            # The first variant we consider is the direct, proportional prioritization where p_i = |δ_i| + eps,
            # where eps is a small positive constant that prevents the edge-case of transitions not being
            # revisited once their error is zero. (Section 3.3)
            priority = (priority + self.eps) ** self.alpha

            self.tree.update(data_idx, priority)
            self.max_priority = max(self.max_priority, priority)

# Create agent
The agent includes 4 main functions:
## train
train function train agent via many episodes:
- Reset the first state.
- loop until the agent wins this stage or reaches the maximum episode/step:
    - predict value, logit for the current state
    - sample action from logit with category distribution (select_action function)
    - log all info to memory
    - train agent every learn_step (on_policy_learn and off_policy_learn functions)
    - eval agent every save_figure_step (save_figure function)
    - set state = next_state (I reset environment when agent reach terminal state then next_state is first state if done=True)

## select_action
this function sample action from logit:
- just convert logit to probability: policy = F.softmax(logits, dim=1)
- create distribution from probability: distribution = torch.distributions.Categorical(policy)
- sample action from distribution: actions = distribution.sample()

## save_figure
this function eval agent and saves agent/video if the agent yields better total rewards:
- reset the environment.
- loop until the agent reaches the terminal state.
    - predict logit from model
    - get action = argmax (logit)
    - environment do this action to get next_state, reward, info, done
    - if total_reward > best test total reward or agent complete this stage, I save model and video.
    - if agent completes this state, we stop training.

## on_policy_learn
this function trains agent from the experiment saved in memory
- get all data from memory
- calculate td (lambda) target and gae advantages
- train num_epoch epochs:
    - shuffle data
    - train with each batch data:
        - calculate loss
        - norm gradient
        - update model from loss
- push data to PER

## off_policy_learn
this function trains agent from the experiment sampled from PER
- sample data from PER
- calculate td (lambda) target and gae advantages
- train 1 epochs:
    - shuffle data
    - train with each batch data:
        - calculate loss
        - norm gradient
        - update model from loss
- update priority for PER

In [ ]:
class Agent():
    def __init__(self, world, stage, action_type, envs, num_envs, state_dim, action_dim, save_dir, save_model_step,
                 save_figure_step, learn_step, total_step_or_episode, total_step, total_episode, model, old_model,
                 gamma, learning_rate, entropy_coef, V_coef, max_grad_norm,
                 clip_param, batch_size, num_epoch, is_normalize_advantage, V_loss_type, target_kl, gae_lambda,
                 per_buffer_size, per_eps, per_alpha, per_beta, eps_marg, off_epoch,
                 device):
        self.world = world
        self.stage = stage
        self.action_type = action_type

        self.state_dim = state_dim
        self.action_dim = action_dim
        self.save_dir = save_dir
        self.learn_step = learn_step
        self.total_step_or_episode = total_step_or_episode
        self.total_step = total_step
        self.total_episode = total_episode

        self.current_step = 0
        self.current_episode = 0

        self.save_model_step = save_model_step
        self.save_figure_step = save_figure_step

        self.device = device
        self.save_dir = save_dir

        self.num_envs = num_envs
        self.envs = envs
        self.model = model.to(self.device)
        self.old_model = old_model.to(self.device)

        self.learning_rate = learning_rate
        self.gamma = gamma
        self.entropy_coef = entropy_coef
        self.V_coef = V_coef
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        self.max_grad_norm = max_grad_norm
        self.clip_param = clip_param
        self.batch_size = batch_size
        self.num_epoch = num_epoch
        self.off_epoch = off_epoch

        self.per_buffer_size = per_buffer_size
        self.per_eps = per_eps
        self.per_alpha = per_alpha
        self.per_beta = per_beta
        self.eps_marg = eps_marg
        self.per = PrioritizedReplayBuffer(self.learn_step, self.state_dim, self.action_dim, self.per_buffer_size, eps=self.per_eps, alpha=self.per_alpha, beta=self.per_beta)
        self.memory = Memory(self.num_envs)
        self.is_completed = False

        self.env = None
        self.max_test_score = -1e9
        self.is_normalize_advantage = is_normalize_advantage
        self.V_loss_type = V_loss_type
        self.target_kl = target_kl
        self.gae_lambda = gae_lambda

        # I just log 1000 lastest update and print it to log.
        self.V_loss = np.zeros((1000,)).reshape(-1)
        self.P_loss = np.zeros((1000,)).reshape(-1)
        self.E_loss = np.zeros((1000,)).reshape(-1)
        self.approx_kl_divs = np.zeros((1000,)).reshape(-1)
        self.total_loss = np.zeros((1000,)).reshape(-1)
        self.loss_index = 0
        self.len_loss = 0

    def save_figure(self, is_training = False):
        # test current model and save model/figure if model yield best total rewards.
        # create env for testing, reset test env
        if self.env is None:
            self.env = create_env(self.world, self.stage, self.action_type, True)
        state = self.env.reset()
        done = False

        images = []
        total_reward = 0
        total_step = 0
        num_repeat_action = 0
        old_action = -1

        episode_time = datetime.now()

        # play 1 episode, just get loop action with max probability from model until the episode end.
        while not done:
            with torch.no_grad():
                logit, V = self.model(torch.tensor(state, dtype = torch.float, device = self.device).unsqueeze(0))
            action = logit.argmax(-1).item()
            next_state, reward, done, trunc, info = self.env.step(action)
            state = next_state
            img = Image.fromarray(self.env.current_state)
            images.append(img)
            total_reward += reward
            total_step += 1

            if action == old_action:
                num_repeat_action += 1
            else:
                num_repeat_action = 0
            old_action = action
            if num_repeat_action == 200:
                break

        #logging, if model yield better result, save figure (test_episode.mp4) and model (best_model.pth)
        if is_training:
            f_out = open(f"logging_test.txt", "a")
            f_out.write(f'episode_reward: {total_reward:.4f} episode_step: {total_step} current_step: {self.current_step} loss_p: {(self.P_loss.sum()/self.len_loss):.4f} loss_v: {(self.V_loss.sum()/self.len_loss):.4f} loss_e: {(self.E_loss.sum()/self.len_loss):.4f} loss: {(self.total_loss.sum()/self.len_loss):.4f} approx_kl_div: {(self.approx_kl_divs.sum()/self.len_loss):.4f} episode_time: {datetime.now() - episode_time}\n')
            f_out.close()

        if total_reward > self.max_test_score or info['flag_get']:
            imageio.mimsave('test_episode.mp4', images)
            self.max_test_score = total_reward
            if is_training:
                torch.save(self.model.state_dict(), f"best_model.pth")

        # if model can complete this game, stop training by set self.is_completed to True
        if info['flag_get']:
            self.is_completed = True

    def save_model(self):
        torch.save(self.model.state_dict(), f"model_{self.current_step}.pth")

    def load_model(self, model_path = None):
        if model_path is None:
            model_path = f"model_{self.current_step}.pth"
        self.model.load_state_dict(torch.load(model_path))

    def update_loss_statis(self, loss_p, loss_v, loss_e, loss, approx_kl_div):
        # update loss for logging, just save 1000 latest updates.
        self.V_loss[self.loss_index] = loss_v
        self.P_loss[self.loss_index] = loss_p
        self.E_loss[self.loss_index] = loss_e
        self.total_loss[self.loss_index] = loss
        self.approx_kl_divs[self.loss_index] = approx_kl_div
        self.loss_index = (self.loss_index + 1)%1000
        self.len_loss = min(self.len_loss+1, 1000)

    def select_action(self, states):
        # select action when training, we need use Categorical distribution to make action base on probability from model
        states = torch.tensor(np.array(states), device = self.device)

        with torch.no_grad():
            logits, V = self.model(states)
            policy = F.softmax(logits, dim=1).clamp(max=1 - 1e-20)
            distribution = torch.distributions.Categorical(policy)
            actions = distribution.sample().cpu().numpy().tolist()
        return actions, logits, V

    def calculate_gae_and_td_target(self, states, actions, next_states, rewards, dones, values = None, b_logits = None):
        if values is None:
            values = [None] * len(states)
            values_ = []
            logits = []
        if b_logits is None:
            b_logits = [None] * len(states)
        else:
            pro_dones = []
            pro_done = 1
        # calculate target (td lambda target) and gae advantages
        targets = []
        with torch.no_grad():
            _, next_value = self.old_model(torch.tensor(np.array(next_states[-1]), device = self.device))
        target = next_value
        advantage = 0
        advantages = []

        for state, action, next_state, reward, done, V, b_logit in zip(states[::-1], actions[::-1], next_states[::-1], rewards[::-1], dones[::-1], values[::-1], b_logits[::-1]):
            if V is None:
                with torch.no_grad():
                    logit, V = self.old_model(torch.tensor(np.array(state), device = self.device))
                    values_.append(V)
                    logits.append(logit)
            if b_logit is not None:
                b_logit = torch.tensor(b_logit, device = self.device)
                prob = torch.softmax(logit, -1).clamp(max=1 - 1e-20)
                b_prob = torch.softmax(b_logit, -1).clamp(max=1 - 1e-20)
                action = torch.tensor(np.array(action).reshape(-1)).long()
                index = torch.arange(len(prob))
                action_prob = prob[index, action]
                action_b_prob = b_prob[index, action]
                done_ = torch.tensor(done, device = self.device, dtype = torch.double).reshape(-1)
                pro_done = torch.ones((1, ), device = self.device).double() * (done_ + (1 - done_) * pro_done) * (action_prob.double() / (action_b_prob + 1e-18).double())
                pro_dones.append(pro_done.clone().clamp(max = 1e15).float())
                pro_done = pro_done.clamp(max = 1e100) #fix when pro_done very small or very large

            done = torch.tensor(done, device = self.device, dtype = torch.float).reshape(-1, 1)
            reward = torch.tensor(reward, device = self.device).reshape(-1, 1)

            target = next_value * self.gamma * (1-done) + reward
            advantage = target + self.gamma * advantage * (1-done) * self.gae_lambda
            targets.append(advantage)
            advantage = advantage - V.detach()
            advantages.append(advantage.detach().view(-1))
            next_value = V.detach()

        targets = targets[::-1]
        advantages = advantages[::-1]

        if b_logits[0] is None:
            if values[0] is None:
                values = values_[::-1]
                logits = logits[::-1]
                return targets, advantages, logits, values, _
            return targets, advantages, _, _, _
        else:
            pro_dones = pro_dones[::-1]
            if values[0] is None:
                values = values_[::-1]
                logits = logits[::-1]
                return targets, advantages, logits, values, pro_dones
            return targets, advantages, _, _, pro_dones


    def on_policy_learn(self):
        # get all data
        states, actions, next_states, rewards, dones, old_logits, old_values = self.memory.get_data()

        # calculate target (td lambda target) and gae advantages
        targets, advantages, _, _, _ = self.calculate_gae_and_td_target(states, actions, next_states, rewards, dones, old_values, None)
        backup_data = (states, actions, next_states, rewards, dones, old_logits, advantages)

        # convert all data to tensor
        action_index = torch.flatten(torch.tensor(actions, device = self.device, dtype = torch.int64))
        states = torch.tensor(np.array(states), device = self.device)
        states = states.reshape((-1,  states.shape[2], states.shape[3], states.shape[4]))
        old_values = torch.cat(old_values, 0)
        targets = torch.cat(targets, 0).view(-1, 1)
        advantages = torch.cat(advantages, 0).view(-1)
        old_logits = torch.cat(old_logits, 0)
        old_probs = torch.softmax(old_logits, -1).clamp(max=1 - 1e-20)
        index = torch.arange(0, len(old_probs), device = self.device)
        old_log_probs = (old_probs[index, action_index] + 1e-9).log()
        early_stopping = False

        #train num_epoch time
        for epoch in range(self.num_epoch):
            #shuffle data for each epoch
            shuffle_ids = torch.randperm(len(targets), dtype = torch.int64)
            for i in range(len(old_values)//self.batch_size):
                #train with batch_size data
                self.optimizer.zero_grad()
                start_id = i * self.batch_size
                end_id = min(len(shuffle_ids), (i+1) * self.batch_size)
                batch_ids = shuffle_ids[start_id:end_id]

                #predict logits and values from model
                logits, values = self.model(states[batch_ids])

                #calculate entropy and value loss (using mse or huber based on config)
                probs =  torch.softmax(logits, -1).clamp(max=1 - 1e-20)
                entropy = (- (probs * (probs + 1e-9).log()).sum(-1)).mean()
                if self.V_loss_type == 'huber':
                    loss_V = F.smooth_l1_loss(values, targets[batch_ids])
                else:
                    loss_V = F.mse_loss(values, targets[batch_ids])

                # calculate log_probs
                index = torch.arange(0, len(probs), device = self.device)
                batch_action_index = action_index[batch_ids]

                log_probs = (probs[index, batch_action_index] + 1e-9).log()

                #approx_kl_div copy from https://stable-baselines3.readthedocs.io/en/master/_modules/stable_baselines3/ppo/ppo.html#PPO
                #if approx_kl_div larger than 1.5 * target_kl (if target_kl in config is not None), stop training because policy change so much
                with torch.no_grad():
                    log_ratio = log_probs - old_log_probs[batch_ids]
                    approx_kl_div = torch.mean((torch.exp(log_ratio) - 1) - log_ratio).cpu().numpy()
                if self.target_kl is not None and approx_kl_div > 1.5 * self.target_kl:
                    early_stopping = True

                #calculate policy loss
                ratio = torch.exp(log_probs - old_log_probs[batch_ids])
                batch_advantages = advantages[batch_ids].detach()
                if self.is_normalize_advantage:
                    batch_advantages = (batch_advantages - batch_advantages.mean()) / (batch_advantages.std() + 1e-9)
                surr1 = ratio * batch_advantages
                surr2 = torch.clamp(ratio, 1.0 - self.clip_param, 1.0 + self.clip_param) * batch_advantages
                loss_P = -torch.min(surr1, surr2).mean()

                # update model
                loss = - entropy * self.entropy_coef + loss_V * self.V_coef + loss_P

                self.update_loss_statis(loss_P.item(), loss_V.item(), entropy.item(), loss.item(), approx_kl_div.item())

                if early_stopping == False:
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
                    self.optimizer.step()
                else:
                    break
            if early_stopping:
                break

        # calculate priority of each trajectory and save it to per
        states, actions, next_states, rewards, dones, old_logits, advantages = backup_data
        priorities = torch.stack(advantages, 0).abs().mean(0)
        states = np.array(states, dtype = np.uint8).transpose(1, 0, 2, 3, 4)
        next_states = np.array(next_states, dtype = np.uint8).transpose(1, 0, 2, 3, 4)
        actions = np.array(actions).transpose(1, 0)
        rewards = np.array(rewards).transpose(1, 0)
        dones = np.array(dones).transpose(1, 0)
        old_logits = torch.stack(old_logits, 1).cpu().numpy()#.transpose(1, 0, 2)
        priorities = priorities.cpu().numpy()
        for state, action, reward, next_state, done, old_logit, priority in zip(states, actions, rewards, next_states, dones, old_logits, priorities):
            self.per.add((state, action, reward, next_state, done, old_logit), priority)

    def off_policy_learn(self):
        # get all data
        (states, actions, next_states, rewards, dones, b_logits), _, per_index = self.per.sample(self.num_envs)

        # calculate target (td lambda target) and gae advantages
        targets, advantages, old_logits, old_values, pro_dones = self.calculate_gae_and_td_target(states, actions, next_states, rewards, dones, None, b_logits)
        backup_data = (states, actions, next_states, rewards, dones, advantages)

        # convert all data to tensor
        action_index = torch.flatten(torch.tensor(actions, device = self.device, dtype = torch.int64))
        states = torch.tensor(np.array(states), device = self.device)
        states = states.reshape((-1,  states.shape[2], states.shape[3], states.shape[4]))
        old_values = torch.cat(old_values, 0)
        targets = torch.cat(targets, 0).view(-1, 1)
        advantages = torch.cat(advantages, 0).view(-1)
        old_logits = torch.cat(old_logits, 0)
        old_probs = torch.softmax(old_logits, -1).clamp(max=1 - 1e-20)
        index = torch.arange(0, len(old_probs), device = self.device)
        old_log_probs = (old_probs[index, action_index] + 1e-9).log()
        early_stopping = False

        b_logits = torch.tensor(b_logits, device = self.device).reshape(-1, b_logits.shape[-1]).float()
        b_probs = torch.softmax(b_logits, -1).clamp(max=1 - 1e-20)
        b_log_probs = (b_probs[index, action_index] + 1e-9).log()
        old_action_probs = old_probs[index, action_index]
        b_action_probs = b_probs[index, action_index]
        pro_dones = torch.cat(pro_dones, 0)
        pro_marg = torch.min(torch.ones(pro_dones.shape, device = self.device) * (1 - self.eps_marg), pro_dones) + torch.relu((pro_dones - (1 - self.eps_marg)) / (pro_dones + 1e-9))
        pro_marg = pro_marg.float()
        advantages = advantages * pro_marg

        #train 1 time
        shuffle_ids = torch.randperm(len(targets), dtype = torch.int64)
        for i in range(len(old_values)//self.batch_size):
                #train with batch_size data
                self.optimizer.zero_grad()
                start_id = i * self.batch_size
                end_id = min(len(shuffle_ids), (i+1) * self.batch_size)
                batch_ids = shuffle_ids[start_id:end_id]

                #predict logits and values from model
                logits, values = self.model(states[batch_ids])

                #calculate entropy and value loss (using mse or huber based on config)
                probs =  torch.softmax(logits, -1).clamp(max=1 - 1e-20)
                entropy = (- (probs * (probs + 1e-9).log()).sum(-1)).mean()

                index = torch.arange(0, len(probs), device = self.device)
                batch_action_index = action_index[batch_ids]
                batch_b_action_probs = b_action_probs[batch_ids].view(-1)
                action_probs = probs[index, batch_action_index].view(-1)

                if self.V_loss_type == 'huber':
                    loss_V =  F.smooth_l1_loss(values, targets[batch_ids], reduction = 'none').view(-1)
                else:
                    loss_V = F.mse_loss(values, targets[batch_ids], reduction = 'none').view(-1)
                loss_V = (pro_marg[batch_ids] * loss_V).mean()

                # calculate log_probs
                log_probs = (probs[index, batch_action_index] + 1e-9).log()

                #approx_kl_div copy from https://stable-baselines3.readthedocs.io/en/master/_modules/stable_baselines3/ppo/ppo.html#PPO
                #if approx_kl_div larger than 1.5 * target_kl (if target_kl in config is not None), stop training because policy change so much
                with torch.no_grad():
                    log_ratio = log_probs - old_log_probs[batch_ids]
                    approx_kl_div = torch.mean((torch.exp(log_ratio) - 1) - log_ratio).cpu().numpy()
                if self.target_kl is not None and approx_kl_div > 1.5 * self.target_kl:
                    early_stopping = True

                #calculate policy loss
                ratio = torch.exp(log_probs - b_log_probs[batch_ids])
                batch_advantages = advantages[batch_ids].detach()
                if self.is_normalize_advantage:
                    batch_advantages = (batch_advantages - batch_advantages.mean()) / (batch_advantages.std() + 1e-9)
                surr1 = ratio * batch_advantages
                surr2 = torch.clamp(ratio, 1.0 - self.clip_param, 1.0 + self.clip_param) * batch_advantages
                loss_P = -torch.min(surr1, surr2).mean()

                # update model
                loss = - entropy * self.entropy_coef + loss_V * self.V_coef + loss_P

                self.update_loss_statis(loss_P.item(), loss_V.item(), entropy.item(), loss.item(), approx_kl_div.item())

                if early_stopping == False:
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
                    self.optimizer.step()
                else:
                    break

        # calculate priority of each trajectory and save it to per
        states, actions, next_states, rewards, dones, advantages = backup_data
        priorities = torch.stack(advantages, 0).abs().mean(0)
        priorities = priorities.cpu().numpy()
        self.per.update_priorities(per_index, priorities)

    def train(self):
        episode_reward = [0] * self.num_envs
        episode_step = [0] * self.num_envs
        max_episode_reward = 0
        max_episode_step = 0
        episode_time = [datetime.now() for _ in range(self.num_envs)]
        total_time = datetime.now()

        last_episode_rewards = []

        #reset envs
        states = self.envs.reset()

        while True:
            # finish training if agent reach total_step or total_episode base on what type of total_step_or_episode is step or episode
            self.current_step += 1

            if self.total_step_or_episode == 'step':
                if self.current_step >= self.total_step:
                    break
            else:
                if self.current_episode >= self.total_episode:
                    break

            actions, logits, values = self.select_action(states)

            next_states, rewards, dones, truncs, infos = self.envs.step(actions)

            # save to memory
            self.memory.save(states, actions, rewards, next_states, dones, logits, values)

            episode_reward = [x + reward for x, reward in zip(episode_reward, rewards)]
            episode_step = [x+1 for x in episode_step]

             # logging after each step, if 1 episode is ending, I will log this to logging.txt
            for i, done in enumerate(dones):
                if done:
                    self.current_episode += 1
                    max_episode_reward = max(max_episode_reward, episode_reward[i])
                    max_episode_step = max(max_episode_step, episode_step[i])
                    last_episode_rewards.append(episode_reward[i])
                    f_out = open(f"logging.txt", "a")
                    f_out.write(f'episode: {self.current_episode} agent: {i} rewards: {episode_reward[i]:.4f} steps: {episode_step[i]} complete: {infos[i]["flag_get"]==True} mean_rewards: {np.array(last_episode_rewards[-min(len(last_episode_rewards), 100):]).mean():.4f} max_rewards: {max_episode_reward:.4f} max_steps: {max_episode_step} current_step: {self.current_step} loss_p: {(self.P_loss.sum()/self.len_loss):.4f} loss_v: {(self.V_loss.sum()/self.len_loss):.4f} loss_e: {(self.E_loss.sum()/self.len_loss):.4f} loss: {(self.total_loss.sum()/self.len_loss):.4f} approx_kl_div: {(self.approx_kl_divs.sum()/self.len_loss):.4f} episode_time: {datetime.now() - episode_time[i]} total_time: {datetime.now() - total_time}\n')
                    f_out.close()
                    episode_reward[i] = 0
                    episode_step[i] = 0
                    episode_time[i] = datetime.now()

            # training agent every learn_step
            if self.current_step % self.learn_step == 0:
                with torch.no_grad():
                    self.old_model.load_state_dict(self.model.state_dict())
                self.on_policy_learn()
                self.memory.reset()
                if self.current_step // self.learn_step >= self.off_epoch:
                    for epoch in range(self.off_epoch):
                        self.off_policy_learn()

            # eval agent every save_figure_step
            if self.current_step % self.save_figure_step == 0:
                self.save_figure(is_training=True)
                if self.is_completed:
                    return

            if self.current_step % self.save_model_step == 0:
                self.save_model()

            states = list(next_states)

        f_out = open(f"logging.txt", "a")
        f_out.write(f' mean_rewards: {np.array(last_episode_rewards[-min(len(last_episode_rewards), 100):]).mean()} max_rewards: {max_episode_reward} max_steps: {max_episode_step} current_step: {self.current_step} total_time: {datetime.now() - total_time}\n')
        f_out.close()

# Create model
I use the same model architecture as [PPO](https://github.com/CVHvn/Mario_PPO/blob/main/src/model.py) without LSTM and init weights.
Model includes:
- 4 convolution layers to encode input image (observation) to feature vector.
- 1 hidden linear layer.
- two linear layers for policy and value prediction (actor and critic).

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(input_dim[0], 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.linear = nn.Linear(1152, 512)
        self.critic_linear = nn.Linear(512, 1)
        self.actor_linear = nn.Linear(512, output_dim)

    def forward(self, x):
        x = F.relu(self.conv1(x/255.))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return self.actor_linear(x), self.critic_linear(x)

model = Model(config.state_dim, config.action_dim)
old_model = Model(config.state_dim, config.action_dim)

# train

In [ ]:
envs = MultipleEnvironments(config.world, config.stage, config.action_type, config.num_envs)

In [ ]:
agent = Agent(world = config.world, stage = config.stage, action_type = config.action_type, envs = envs, num_envs = config.num_envs,
              state_dim = config.state_dim, action_dim = config.action_dim, save_dir = config.save_dir,
              save_model_step = config.save_model_step, save_figure_step = config.save_figure_step, learn_step = config.learn_step,
              total_step_or_episode = config.total_step_or_episode, total_step = config.total_step, total_episode = config.total_episode,
              model = model, old_model = old_model, gamma = config.gamma, learning_rate = config.learning_rate,
              entropy_coef = config.entropy_coef, V_coef = config.V_coef,
              max_grad_norm = config.max_grad_norm, clip_param = config.clip_param, batch_size = config.batch_size,
              num_epoch = config.num_epoch, is_normalize_advantage = config.is_normalize_advantage, V_loss_type = config.V_loss_type,
              target_kl = config.target_kl, gae_lambda = config.gae_lambda,  per_buffer_size = config.per_buffer_size,
              per_eps = config.per_eps, per_alpha = config.per_alpha, per_beta = config.per_beta, eps_marg = config.eps_marg, off_epoch = config.off_epoch,
              device = "cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
agent.train()

#test

In [ ]:
agent.load_model("best_model.pth")
agent.save_figure()